In [1]:
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
from sb3_contrib.ppo_mask import MaskablePPO
import numpy as np
import wandb
import gym
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'sb3.ipynb'
wandb.login()

wandb: Currently logged in as: hojmax (rl-msps). Use `wandb login --relogin` to force relogin


True

In [2]:
config = {
    'total_timesteps': 25000,
}

In [3]:
run = wandb.init(
    project="Q-learning",
    entity="rl-msps",
    sync_tensorboard=True,
    name=f"Cartpole-SB3",
    config=config,
    tags=["test"]
)

In [4]:
def mask_fn(env):
    return np.ones(env.action_space.n)

env = gym.make('CartPole-v1')
env = Monitor(env)
env = ActionMasker(env, mask_fn)

In [5]:
model = MaskablePPO(
    MaskableActorCriticPolicy,
    env,
    verbose=1,
    tensorboard_log=f"runs/{run.id}"
)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [7]:
model.learn(
    total_timesteps=config['total_timesteps'],
    callback=WandbCallback(
        model_save_path=f"models/{run.id}",
    )
)

Logging to runs/3i4b4gdl/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 25       |
|    ep_rew_mean     | 25       |
| time/              |          |
|    fps             | 2353     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30          |
|    ep_rew_mean          | 30          |
| time/                   |             |
|    fps                  | 1391        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008043068 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00242     |

In [8]:
wandb.finish()

global_step,▁▂▂▃▃▄▅▅▆▆▇▇█
rollout/ep_len_mean,▁▁▁▂▂▃▄▄▅▆▇▇█
rollout/ep_rew_mean,▁▁▁▂▂▃▄▄▅▆▇▇█
time/fps,█▁▁▁▁▂▂▁▁▁▁▂▂
train/approx_kl,▆███▄▅▃▃▁▄▁▂
train/clip_fraction,█▅▇▅▄▅▃▃▁▃▁▂
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▁▂▄▅▆▆▆▇▇▇██
train/explained_variance,▁▂▃▃▄▆▇▆▆▂▆█
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▂▄▇█▄▂▂▅▂▂▅▁
